### Weiqing Kong

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from collections import Counter

In [2]:
sheet_id = '16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM'
sheet_name = 'Shopify'
URL = 'https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(sheet_id,sheet_name)

In [3]:
df = pd.read_csv(URL);df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


In [4]:
df['order_amount'].mean()

3145.128

In [5]:
def outlier_dect(x,y):
    q1 = np.quantile(x,0.25)
    q3 = np.quantile(x,0.75)
    mean = np.mean(x)
    std = np.std(x)
    IQR = q3 - q1
    if y == 'z_test':
        outlier = [j for i,j in enumerate(x) if scipy.stats.norm.sf(abs((j-mean)/std)) < 0.05 ]
        outlier_index = [i for i,j in enumerate(x) if scipy.stats.norm.sf(abs((j-mean)/std)) < 0.05]
    if y == 'quantile':
        outlier = [j for i,j in enumerate(x) if (j > q3 + 1.5*IQR) or (j < q1 - 1.5*IQR)]
        outlier_index = [i for i,j in enumerate(x) if (j > q3 + 1.5*IQR) or (j < q1 - 1.5*IQR)]
    return outlier, outlier_index

In [6]:
df['single_price'] = df['order_amount']/df['total_items'];df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,single_price
0,1,53,746,224,2,cash,2017-03-13 12:36:56,112.0
1,2,92,925,90,1,cash,2017-03-03 17:38:52,90.0
2,3,44,861,144,1,cash,2017-03-14 4:23:56,144.0
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,156.0
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,156.0
...,...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17,165.0
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16,117.0
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42,117.0
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18,177.0


In [7]:
#calculate each shop's single price of selling sneakers and verify the correctness of data
shop_id = np.unique(df['shop_id'])
shop_price = [[i] for i in shop_id]
for i,j in enumerate(df['shop_id']):
    shop_price[j-1].append(df['single_price'][i])
shop_price = [np.unique(i) for i in shop_price]
shop_price

[array([  1., 158.]),
 array([ 2., 94.]),
 array([  3., 148.]),
 array([  4., 128.]),
 array([  5., 142.]),
 array([  6., 187.]),
 array([  7., 112.]),
 array([  8., 132.]),
 array([  9., 118.]),
 array([ 10., 148.]),
 array([ 11., 184.]),
 array([ 12., 201.]),
 array([ 13., 160.]),
 array([ 14., 116.]),
 array([ 15., 153.]),
 array([ 16., 156.]),
 array([ 17., 176.]),
 array([ 18., 156.]),
 array([ 19., 163.]),
 array([ 20., 127.]),
 array([ 21., 142.]),
 array([ 22., 146.]),
 array([ 23., 156.]),
 array([ 24., 140.]),
 array([ 25., 130.]),
 array([ 26., 176.]),
 array([ 27., 169.]),
 array([ 28., 164.]),
 array([ 29., 163.]),
 array([ 30., 153.]),
 array([ 31., 129.]),
 array([ 32., 101.]),
 array([ 33., 173.]),
 array([ 34., 122.]),
 array([ 35., 164.]),
 array([ 36., 130.]),
 array([ 37., 142.]),
 array([ 38., 190.]),
 array([ 39., 134.]),
 array([ 40., 161.]),
 array([ 41., 118.]),
 array([ 42., 352.]),
 array([ 43., 181.]),
 array([ 44., 144.]),
 array([ 45., 142.]),
 array([ 46.

In [9]:
price = [i[1] for i in shop_price]
outlier_dect(price, 'quantile')

([352.0, 25725.0], [41, 77])

The outliers here are shop_id 42 with a single price 352 and shop_id 78 with a single price 25725.

In [41]:
price_outlier_index = list(np.where(df['shop_id'] == 42)[0]) + list(np.where(df['shop_id'] == 78)[0])

In [18]:
np.where(df['shop_id'] == 78)

(array([ 160,  490,  493,  511,  617,  691, 1056, 1193, 1204, 1259, 1384,
        1419, 1452, 1529, 2270, 2452, 2492, 2495, 2512, 2548, 2564, 2690,
        2773, 2818, 2821, 2906, 2922, 3085, 3101, 3151, 3167, 3403, 3440,
        3705, 3724, 3780, 4040, 4079, 4192, 4311, 4412, 4420, 4505, 4584,
        4715, 4918], dtype=int64),)

In [42]:
amount_outlier, amount_outlier_index = outlier_dect(df['order_amount'],'quantile')

In [45]:
outlier_index = np.unique(price_outlier_index + amount_outlier_index);outlier_index

array([  15,   40,   60,   99,  136,  160,  219,  222,  259,  264,  308,
        409,  490,  493,  511,  520,  522,  617,  651,  691,  737,  742,
        771,  834,  835,  879,  938,  979,  994, 1056, 1104, 1123, 1150,
       1193, 1204, 1256, 1259, 1362, 1364, 1367, 1384, 1419, 1436, 1452,
       1471, 1484, 1503, 1512, 1520, 1529, 1562, 1563, 1602, 1628, 1764,
       1911, 1923, 1929, 1946, 1948, 1962, 2003, 2018, 2032, 2039, 2043,
       2053, 2127, 2136, 2140, 2153, 2270, 2273, 2297, 2307, 2353, 2386,
       2389, 2452, 2491, 2492, 2494, 2495, 2512, 2548, 2560, 2564, 2609,
       2670, 2690, 2757, 2766, 2773, 2818, 2821, 2835, 2906, 2922, 2967,
       2969, 2987, 3073, 3077, 3085, 3101, 3117, 3151, 3167, 3202, 3252,
       3332, 3403, 3428, 3438, 3440, 3513, 3517, 3532, 3538, 3609, 3651,
       3697, 3705, 3724, 3780, 3865, 3903, 3927, 3966, 3998, 4040, 4056,
       4079, 4141, 4192, 4231, 4294, 4295, 4311, 4326, 4412, 4420, 4421,
       4490, 4505, 4512, 4523, 4554, 4574, 4580, 45

In [53]:
df1 = df.copy()
df_outlier_dropped = df1.drop(index = outlier_index)
df_outlier_dropped.index = list(np.arange(df_outlier_dropped.shape[0]));df_outlier_dropped

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,single_price
0,1,53,746,224,2,cash,2017-03-13 12:36:56,112.0
1,2,92,925,90,1,cash,2017-03-03 17:38:52,90.0
2,3,44,861,144,1,cash,2017-03-14 4:23:56,144.0
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,156.0
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,156.0
...,...,...,...,...,...,...,...,...
4826,4996,73,993,330,2,debit,2017-03-30 13:47:17,165.0
4827,4997,48,789,234,2,cash,2017-03-16 20:36:16,117.0
4828,4998,56,867,351,3,cash,2017-03-19 5:42:42,117.0
4829,4999,60,825,354,2,credit_card,2017-03-16 14:51:18,177.0


In [55]:
aov = np.mean(df_outlier_dropped['order_amount']);aov

292.4303456841234

This is where the number 3145.13 comes from. It is calulated directly by averaging the order amount of each line. The result looks inreasonable because, there are some outliers exist. By the outlier detection function with z-test and quantile methods, the function can output those outliers that are much larger than other order amounts due to either large 'total_items' values or high single price.

The metric used above is to conclude outliers in two different ways:<br>
1) Caluclate single item price for each shop and exclude those shops that have unusual single item prices. <br>
The result are the shop_id 42 with a single price 352 and shop_id 78 with a single price 25725.<br>
2) Use outlier detection function to find outliers in the column 'order_amount' that could include some other unusual orders from shops apart from the shop 42 and the shop 78. <br>
The result used the quantile method instead of z-test hypothesis because, the significance level is diffcult to determine - even the result computed using 10% significance include large amount orders.  

By dropping the outliers discussed above, the final AOV value is 292.43.